In [ ]:
from owlready2 import *
import datetime
import re
import logging
import editdistance
from pprint import pprint
from nltk.corpus import wordnet as wn
from __init__ import *

In [ ]:
# ontology setup
onto = get_ontology("resources/ISTAT_RDF_XML.OWL")
onto.load()
sync_reasoner()

In [ ]:
# logger setup
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
#1

domande = []

for c in onto.classes():
    
    # i verbi sono preceduti da "che" nelle object properties?
    
    # controlla subclasses
    for c2 in c.ancestors():
        if c2.name != "Thing" and c2.name != c.name:
            question_string = c2.name.replace("_", " ") + " che è " + c.name.replace("_", " ")
            domande.append(question_string)
            
            if c2.name in c.name:
                question_string = c2.name.replace("_", " ") + " che è " + c.name.replace(c2.name, "").replace("_", " ")
                question_string = re.sub("\s{2,}", " ", question_string)
                domande.append(question_string)
    
    # controlla object properties
    for p in onto.object_properties():
        
        if c in p.domain:
            
            question_string = ""
            question_string += c.name.replace("_", " ") + " "
            question_string += p.name.replace("_", " ") + " "
            
            for r in p.range:
                question_string += "[" + r.name + "]"
            
            domande.append(question_string)
            
    # controlla data properties
    for p in onto.data_properties():
        
        if c in p.domain:
            
            question_string = ""
            question_string += c.name.replace("_", " ") + " con "
            question_string += p.name.replace("_", " ") + " "
            
            for r in p.range:
                
                if r == str:
                    question_string += "[STR]"
                elif r == int:
                    question_string += "[INT]"
                elif r == datetime.datetime:
                    question_string += "[DATE]"
                    
            domande.append(question_string)
            
logger.debug("created " + str(len(domande)) + " questions")
for d in domande:
    print(d)

In [ ]:
from lib.treetagger import TreeTagger

tt = TreeTagger(language='italian')
tt.tag('lupa')

In [ ]:
find_most_similar(domande, "persona che è cittadino")
preprocess_string("persona che è cittadino")

In [ ]:
generate_questions(onto)

In [ ]:
for i, c in enumerate(onto.classes()):
    #pprint(vars(c))
    if i==4:
        print(vars(c))
        
for c in onto.disjoint_classes():
    try:
        print(c.entities[0].name + "\t" + c.entities[1].name)
    except:
        print("error")

In [37]:
def isa_dfs(onto, start, visited=None):
    
    # mancano gli isa con qualche restrizione?
    if visited is None:
        visited=set()
    visited.add(start)
    for next in start.ancestors() - visited:
        isa_dfs(onto, next, visited)
    for next in start.descendants() - visited:
        isa_dfs(onto, next, visited)
    return visited


def find_all_isa_sets(onto):
    
    sets = set()

    for c in onto.classes():
        new_set = isa_dfs(onto, c, None)
        if new_set not in sets:
            #print(new_set)
            sets.add(frozenset(new_set))
    return sets


def generate_impossible_combinations(onto):
    for disj in onto.disjoint_classes():
        try:
            for elem in disj.entities[0].descendants():
                for elem2 in disj.entities[1].descendants():
                    print(elem.name + " " + elem2.name)
        except:
            pass

def remove_sets_containing_disjoint_classes(onto, subsets):
    """
    result = set()
    
    for subset in subsets:
        valid = True
        for i in onto.disjoint_classes():
            try:
                if i.entities[0] in subset and i.entities[1] in subset:
                    valid = False
                    break
                    
                subset_list = list(subset)
                for i, elem in enumerate(subset_list):
                    if elem in i.entities[0].descendants():
                        for elem2 in subset_list:
                            if elem2 != elem:
                                if elem2 in i.entities[1].descendants():
                                    valid = False
                                    break
            except:
                pass
        if valid:
            result.add(subset)
            
    return result
    """
    results = set()
    
    for disj in onto.disjoint_classes():
        set_disj = set()
        try:
            for elem in disj.entities:
                set_disj.add(elem)
        except:
            pass
        for subset in subsets:
            if len(set_disj.intersection(subset)) > 2:
                print(set_disj.intersection(subset))
    
    return results
    

def generate_isa_questions(onto, check_disjointness=True):
    
    import itertools
    
    count =0
    
    for s in find_all_isa_sets(onto):
        for l in range(0, len(s)+1):
            subsets = itertools.combinations(s, l)
            subsets = remove_sets_containing_disjoint_classes(onto, subsets)
            for subset in subsets:
                st = ""
                for i, elem in enumerate(subset):
                    if elem.name is not "Thing":
                        name = elem.name.replace("_", " ").lower()
                        if i<len(subset)-1: 
                            st += name + " e "
                        else:
                            st += name + " "
                #print(st)
                count += 1

        print("")
        
    print(count)
    
generate_isa_questions(onto)
#generate_impossible_combinations(onto)

{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario}
{ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Fratello_sorella_dello_intestatario}
{ISTAT_R

{ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Cognato_a_dello_intestatario, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.

{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT

{ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Cognato_a_dello_intestatario, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Cognato_a_dello_intestatario, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_con

{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario, ISTAT_RDF_XML.OWL.Ignoto}
{ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestata

{ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario, ISTAT_RDF_XML.OWL.Fratello_sorella_dello_intestatario, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario}
{ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario, ISTAT_RDF_XML.OWL.Fratello_sorella_dello_intestatario, ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato}
{ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XM

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [22]:
#print(vars(onto.Altro_legame_con_lo_intestatario_non_altrove_specificato))
for a in onto.disjoint_classes():
    print(a)

AllDisjoint([ISTAT_RDF_XML.OWL.Altro_legame_con_lo_intestatario_non_altrove_specificato, ISTAT_RDF_XML.OWL.Cognato_a_dello_intestatario, ISTAT_RDF_XML.OWL.Coniuge_o_convivente_dello_intestatario, ISTAT_RDF_XML.OWL.Figlio_intestatario_e_del_coniuge_convivente, ISTAT_RDF_XML.OWL.Fratello_sorella_dello_intestatario, ISTAT_RDF_XML.OWL.Genero_nuora_coniuge_convivente_del_figlio_dello_intestatario, ISTAT_RDF_XML.OWL.Genitore_o_coniuge_convivente_del_genitore_di_intestatario, ISTAT_RDF_XML.OWL.Ignoto, ISTAT_RDF_XML.OWL.Intestatario_foglio_famiglia, ISTAT_RDF_XML.OWL.Nipote_dello_intestatario, ISTAT_RDF_XML.OWL.Nonno_a_dello_intestatario, ISTAT_RDF_XML.OWL.Suocero_a_dello_intestatario])
AllDisjoint([ISTAT_RDF_XML.OWL.Area_censimento, ISTAT_RDF_XML.OWL.Localita, ISTAT_RDF_XML.OWL.Sezione_censimento])
AllDisjoint([ISTAT_RDF_XML.OWL.Area_speciale, ISTAT_RDF_XML.OWL.Case_sparse, ISTAT_RDF_XML.OWL.Entita_amministrativa, ISTAT_RDF_XML.OWL.Entita_statistico_geografica, ISTAT_RDF_XML.OWL.Famiglia_anag

TypeError: 'NoneType' object is not subscriptable